In [1]:
import os
import pandas as pd
import numpy as np
import time

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

In [2]:
#create key for the google sheet
SCOPES = ["https://www.googleapis.com/auth/spreadsheets"]
SPREADSHEET_ID = ["spreadsheet_id"]

In [3]:
#You can add cleaning process here according to your need of the data you have

In [4]:
#DEFINING SECTION ANALISIS

kebekerjaan = ['list of questions']
kewirausahaan = ['list of questions']
manfaat_pelatihan = ['list of questions']
evaluasi_sikap = ['list of questions']
kesejahteraan = ['list of questions']
inklusi_keuangan = ['list of questions']
pmi = ['list of questions']
persepsi_pelatihan = ['list of questions']

In [5]:
#function to define google sheets API credential
def main():
    credentials = None
    if os.path.exists("token.json"):
        credentials = Credentials.from_authorized_user_file("token.json", SCOPES)
    if not credentials or not credentials.valid:
        if credentials and credentials.expired and credentials.refresh_token:
            credentials.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
            credentials = flow.run_local_server(port=0)
        with open("token.json", "w") as token:
            token.write(credentials.to_json())
    
    return credentials

#function to call google sheet in your python
def build_service():
    service = build('sheets', 'v4', credentials=credentials)
    return service

def number_analysis(df, i):
    mean = df[i].mean()
    median = df[i].median()
    summary_dict = {'mean': mean, 'median': median}
    df_x = pd.DataFrame(summary_dict.values(), index=summary_dict.keys(), columns=[i])
    return df_x

#define number of the cell you will to paste your analysis from python to goole sheet
def set_cells(section, sheet):
    space_cells_raw = []
    space_cells = [1]
    index = 0

    while index < len(section):
        for i in section:
            if i in df_se1.columns:
                #if datatype are integer or float
                if df_se1[i].dtypes in ['float64', 'int64']:
                    df_x = number_analysis(df_se1, i)
                    space_cells_raw.append(len(df_x) + 4)
                #if datatype is object
                else:
                    df_x = pd.DataFrame(df_se1[i].value_counts())
                    space_cells_raw.append(len(df_x) + 4)
            elif i in df_se2.columns:
                if df_se2[i].dtypes in ['float64', 'int64']:
                    df_x = number_analysis(df_se2, i)
                    space_cells_raw.append(len(df_x) + 4)
                #if datatype is object
                else:
                    df_x = pd.DataFrame(df_se2[i].value_counts())
                    space_cells_raw.append(len(df_x) + 4)          
        if (index >= 0):
            space_cells.append(space_cells_raw[index] + space_cells[-1])
            space_cells_raw.append(space_cells_raw)
        index += 1
    return space_cells

#function to append data to your google sheet
def append_to_sheet(service, spreadsheet_id, sheet, row, data):
    retry_count = 3
    
    for retry in range(retry_count):
        try:
            service.spreadsheets().values().append(
            spreadsheetId = spreadsheet_id,
            valueInputOption = "RAW",
            range = f"{sheet}!A{row}",
            body = {"values": data}).execute()
            
            break
            
        except HttpError as error:
            if error.resp.status == 429:
                print(f"Rate limit exceeded. Retrying in 20 seconds. Retry {retry + 1}/{retry_count}")
                time.sleep(20)
            else:
                print(f"An error occurred: {error}")
                break
            
#function to data analysis through looping
def analysis(section, sheet, data1, data2):
    data1 = 'data 1 directory'
    data2 = 'data 2 directory'
    
    df_se1 = pd.read_stata(data1)
    df_se2 = pd.read_stata(data2)
    
    analysis_section = section
        
    range_cells = set_cells(section, sheet)
    
    for i, row in zip(analysis_section, range_cells):
        if i in df_se1.columns:
                #if datatype are integer or float
            if df_se1[i].dtypes in ['float64', 'int64']:
                df_x = number_analysis(df_se1, i)
                #if datatype is object
            else:
                df_x = pd.DataFrame(df_se1[i].value_counts())
            
        elif i in df_se2.columns:
            if df_se2[i].dtypes in ['float64', 'int64']:
                df_x = number_analysis(df_se1, i)
                #if datatype is object
            else:
                df_x = pd.DataFrame(df_se2[i].value_counts())
            
        try:
            append_to_sheet(service, SPREADSHEET_ID, sheet, row, df_x.T.reset_index().T.values.tolist())            
        except Exception as e:
            print(f"Failed to append data for {i}: {str(e)}")

In [ ]:
credentials = main()
service = build_service()
analysis(name_section, 'name_sheet', 'file_directory_data1', 'file_directory_data2')